In [2]:
import brian2 as b2
from brian2 import NeuronGroup, Synapses, PoissonInput, PoissonGroup, network_operation
from brian2.monitors import StateMonitor, SpikeMonitor, PopulationRateMonitor
from random import sample
import numpy.random as rnd
from neurodynex3.tools import plot_tools
import numpy
import matplotlib.pyplot as plt
from math import floor
import time

b2.defaultclock.dt = 0.10 * b2.ms

from brian2 import *




In [7]:
def nmda_by_x(x_rate, w_pos):
    
    start_scope()
    #w_pos = 1.9
    N_Excit=384
    N_Inhib=96
    weight_scaling_factor=5.33
    t_stimulus_start=100 * b2.ms
    t_stimulus_duration=9999 * b2.ms
    coherence_level=0.
    stimulus_update_interval=30 * b2.ms
    mu0_mean_stimulus_Hz=250.
    stimulus_std_Hz=20.
    N_extern=1000
    firing_rate_extern=9.8 * b2.Hz
    f_Subpop_size=0.25  # .15 in publication [1]                     
    max_sim_time=1000. * b2.ms 
    stop_condition_rate=None

    monitored_subset_size=512, 
    E_leak_excit = -70.0 * b2.mV

    print("simulating {} neurons. Start: {}".format(N_Excit + N_Inhib, time.ctime()))
    t_stimulus_end = t_stimulus_start + t_stimulus_duration

    N_Group_A = int(N_Excit * f_Subpop_size)  # size of the excitatory subpopulation sensitive to stimulus A
    N_Group_B = N_Group_A  # size of the excitatory subpopulation sensitive to stimulus B
    N_Group_Z = N_Excit - N_Group_A - N_Group_B  # (1-2f)Ne excitatory neurons do not respond to either stimulus.

    Cm_excit = 0.5 * b2.nF  # membrane capacitance of excitatory neurons
    G_leak_excit = 25.0 * b2.nS  # leak conductance
    E_leak_excit = -70.0 * b2.mV  # reversal potential   #######################################################
    v_spike_thr_excit = -50.0 * b2.mV  # spike condition
    v_reset_excit = -60.0 * b2.mV  # reset voltage after spike
    t_abs_refract_excit = 2. * b2.ms  # absolute refractory period

    # specify the inhibitory interneurons:
    # N_Inhib = 200
    Cm_inhib = 0.2 * b2.nF
    G_leak_inhib = 20.0 * b2.nS
    E_leak_inhib = -90.0 * b2.mV
    v_spike_thr_inhib = -50.0 * b2.mV
    v_reset_inhib = -60.0 * b2.mV
    t_abs_refract_inhib = 1.0 * b2.ms

    # specify the AMPA synapses
    E_AMPA = 0.0 * b2.mV
    tau_AMPA = 2.5 * b2.ms

    # specify the GABA synapses
    E_GABA = -70.0 * b2.mV
    tau_GABA = 5.0 * b2.ms

    # specify the NMDA synapses
    E_NMDA = 0.0 * b2.mV
    tau_NMDA_s = 100.0 * b2.ms
    tau_NMDA_x = 2. * b2.ms
    alpha_NMDA = 0.5 * b2.kHz

    # projections from the external population
    g_AMPA_extern2inhib = 1.62 * b2.nS
    g_AMPA_extern2excit = 2.1 * b2.nS

    # projectsions from the inhibitory populations
    g_GABA_inhib2inhib = weight_scaling_factor * 1.25 * b2.nS
    g_GABA_inhib2excit = weight_scaling_factor * 1.60 * b2.nS

    # projections from the excitatory population
    g_AMPA_excit2excit = weight_scaling_factor * 0.012 * b2.nS
    g_AMPA_excit2inhib = weight_scaling_factor * 0.015 * b2.nS
    g_NMDA_excit2excit = weight_scaling_factor * 0.040 * b2.nS
    g_NMDA_excit2inhib = weight_scaling_factor * 0.045 * b2.nS  # stronger projection to inhib.

    # weights and "adjusted" weights.
    w_neg = 1. - f_Subpop_size * (w_pos - 1.) / (1. - f_Subpop_size)
    # We use the same postsyn AMPA and NMDA conductances. Adjust the weights coming from different sources:
    w_ext2inhib = g_AMPA_extern2inhib / g_AMPA_excit2inhib
    w_ext2excit = g_AMPA_extern2excit / g_AMPA_excit2excit
    # other weights are 1
    # print("w_neg={}, w_ext2inhib={}, w_ext2excit={}".format(w_neg, w_ext2inhib, w_ext2excit))A
    # Define the inhibitory population



    excit_lif_dynamics = """
            s_NMDA_total : 1  # the post synaptic sum of s. compare with s_NMDA_presyn
            dv/dt = (
            - G_leak_excit * (v-E_leak_excit)
            - g_AMPA_excit2excit * s_AMPA * (v-E_AMPA)
            - g_GABA_inhib2excit * s_GABA * (v-E_GABA)
            - g_NMDA_excit2excit * s_NMDA_total * (v-E_NMDA)/(1.0+1.0*exp(-0.062*v/volt)/3.57)
            )/Cm_excit : volt (unless refractory)
            ds_AMPA/dt = -s_AMPA/tau_AMPA : 1
            ds_GABA/dt = -s_GABA/tau_GABA : 1
            ds_NMDA/dt = -s_NMDA/tau_NMDA_s + alpha_NMDA * x * (1-s_NMDA) : 1
            dx/dt = -x/tau_NMDA_x : 1
        """
    
    excit_lif_dynamics_nmda = """
            s_NMDA_total : 1  # the post synaptic sum of s. compare with s_NMDA_presyn
            dv/dt = (
            - G_leak_excit * (v-E_leak_excit)
            - g_AMPA_excit2excit * s_AMPA * (v-E_AMPA)
            - g_GABA_inhib2excit * s_GABA * (v-E_GABA)
            - g_NMDA_excit2excit * s_NMDA_total * (v-E_NMDA)/(1.0+1.0*exp(-0.062*v/volt)/3.57)
            )/Cm_excit : volt (unless refractory)
            ds_AMPA/dt = -s_AMPA/tau_AMPA : 1
            ds_GABA/dt = -s_GABA/tau_GABA : 1
            ds_NMDA/dt = -s_NMDA/tau_NMDA_s + alpha_NMDA * x * (1-s_NMDA) : 1
            dx/dt = -x/tau_NMDA_x : 1
        """


    one_neuron = NeuronGroup(1, model=excit_lif_dynamics_nmda,
                            threshold="v>v_spike_thr_excit", reset="v=v_reset_excit",
                            refractory=t_abs_refract_excit, method="rk2")

    one_neuron.v = rnd.uniform(E_leak_excit / b2.mV, high=E_leak_excit / b2.mV + 5., size=one_neuron.N) * b2.mV




    # network parameters
    N_E = 1000
    gamma = 0.25
    N_I = round(gamma * N_E)
    N = N_E + N_I
    epsilon = 0.1
    C_E = epsilon * N_E
    C_ext = C_E

    # neuron parameters
    tau = 10 * ms
    theta = 1 * mV
    V_r = 0 * mV
    tau_rp = 2 * ms

    # synapse parameters
    J = 0.1 * mV
    D = 1.5 * ms
    nu_ext = 10 *Hz
    # external stimulus
    nu_thr = theta / (J * C_E * tau)

    defaultclock.dt = 0.1 * ms

    Poisson_NE = 1000
    g = 1

    #w_pos= 1.9



    # with and without NMDA 와 비교하기 
    exc_poisson_input = PoissonInput(
            target=one_neuron, target_var="s_AMPA", N=N_E, rate=1*nu_ext, weight= w_ext2excit)       

    inh_poisson_input = PoissonInput(
            target=one_neuron, target_var="s_GABA", N=N_I, rate=1*nu_ext, weight= w_ext2inhib)

    nmda_poisson_input = PoissonInput(
            target=one_neuron, target_var="x", N=1, rate= x_rate*Hz, weight= 1 )

    #NDMA input 
    sNMDA_A_total = []

    @network_operation()
    def update_nmda_sum():
            sum_sNMDA_A = 300*sum(one_neuron.s_NMDA)

            #sum_sNMDA_A = 100

            # note the _ at the end of s_NMDA_total_ disables unit checking
            one_neuron.s_NMDA_total_ = (w_pos * sum_sNMDA_A)  ## 자기 자신의 nmda input 만을 받는다. 
            sNMDA_A_total.append(sum_sNMDA_A)


    # set a self-recurrent synapse to introduce a delay when updating the intermediate
    # gating variable x

    syn_x_A2A = Synapses(one_neuron, one_neuron, on_pre="x += 1.", delay=0.5 * b2.ms)
    syn_x_A2A.connect(j="i")

    rmp_monitor = StateMonitor(one_neuron, 'v', record = 0)

    x_monitor = StateMonitor(one_neuron, 'x', record = 0)
    s_monitor = StateMonitor(one_neuron, 's_NMDA', record = 0)

    rmp_in_mV = 1000*(rmp_monitor.v[0])
    run(1200*ms, report='text')

    time_in_ms = (rmp_monitor.t/ms)/1000
    rmp_in_mV = 1000*(rmp_monitor.v[0])
    result = {}
    
    result['time'] = time_in_ms
    result['rmp'] = rmp_in_mV/volt
    
    
    return result

In [39]:
rmp_values_total = {}



In [40]:
rmp_w_pos = []



In [41]:
w_pos_level_raw = numpy.arange(1,2.5,0.1)

In [42]:
w_pos_level = numpy.round(w_pos_level_raw, 1)

In [53]:
for w_pos in w_pos_level[:2]:
    
    rmp_w_pos = []
    
    for trial_n in range(2):
        
        rmp_w_pos.append(nmda_by_x(100, w_pos))
        
    rmp_values_total[w_pos] = rmp_w_pos
    

simulating 480 neurons. Start: Fri Nov  4 15:12:38 2022
Starting simulation at t=0. s for a duration of 1.2 s
1.2 s (100%) simulated in 1s
simulating 480 neurons. Start: Fri Nov  4 15:12:40 2022
Starting simulation at t=0. s for a duration of 1.2 s
1.2 s (100%) simulated in 1s
simulating 480 neurons. Start: Fri Nov  4 15:12:42 2022
Starting simulation at t=0. s for a duration of 1.2 s
1.2 s (100%) simulated in 1s
simulating 480 neurons. Start: Fri Nov  4 15:12:44 2022
Starting simulation at t=0. s for a duration of 1.2 s
1.2 s (100%) simulated in 1s


In [54]:
rmp_values_total

{1.1: [{'time': array([0.0000e+00, 1.0000e-04, 2.0000e-04, ..., 1.1997e+00, 1.1998e+00,
          1.1999e+00]),
   'rmp': array([-68.94352778, -68.94879694, -68.86916942, ..., -66.93591369,
          -67.02998801, -67.06745811])},
  {'time': array([0.0000e+00, 1.0000e-04, 2.0000e-04, ..., 1.1997e+00, 1.1998e+00,
          1.1999e+00]),
   'rmp': array([-65.92207642, -65.94241506, -65.9085214 , ..., -66.4765598 ,
          -66.51173531, -66.51396914])}],
 1.0: [{'time': array([0.0000e+00, 1.0000e-04, 2.0000e-04, ..., 1.1997e+00, 1.1998e+00,
          1.1999e+00]),
   'rmp': array([-65.16675614, -65.19086195, -65.16133195, ..., -67.39672067,
          -67.38249552, -67.35620081])},
  {'time': array([0.0000e+00, 1.0000e-04, 2.0000e-04, ..., 1.1997e+00, 1.1998e+00,
          1.1999e+00]),
   'rmp': array([-69.4235641 , -69.42643908, -69.37231649, ..., -66.8419957 ,
          -66.74569835, -66.6634023 ])}]}

In [43]:
for trial_n in range(2):
    
    rmp_w_pos = []
    
    for w_pos in w_pos_level[:2]:
        
        rmp_w_pos.append(nmda_by_x(100, w_pos))
    
    rmp_values_total[w_pos] = rmp_w_pos
    

simulating 480 neurons. Start: Fri Nov  4 15:09:25 2022
Starting simulation at t=0. s for a duration of 1.2 s
1.2 s (100%) simulated in 1s
simulating 480 neurons. Start: Fri Nov  4 15:09:27 2022
Starting simulation at t=0. s for a duration of 1.2 s
1.2 s (100%) simulated in 1s
simulating 480 neurons. Start: Fri Nov  4 15:09:29 2022
Starting simulation at t=0. s for a duration of 1.2 s
1.2 s (100%) simulated in 1s
simulating 480 neurons. Start: Fri Nov  4 15:09:31 2022
Starting simulation at t=0. s for a duration of 1.2 s
1.2 s (100%) simulated in 1s


In [45]:
w_pos_level[:2]

array([1. , 1.1])

In [48]:
rmp_values_total.keys()

dict_keys([1.1])

In [ ]:
rmp_w_pos

In [8]:
nmda_by_x(100, 1)

simulating 480 neurons. Start: Fri Nov  4 14:53:00 2022
Starting simulation at t=0. s for a duration of 1.2 s
1.2 s (100%) simulated in 1s


{'time': array([0.0000e+00, 1.0000e-04, 2.0000e-04, ..., 1.1997e+00, 1.1998e+00,
        1.1999e+00]),
 'rmp': array([-65.35000548, -65.37319732, -65.36943542, ..., -67.06107953,
        -66.97151483, -66.87056097])}

In [17]:
trial_n

9

In [ ]:
rmp_values_total()